In [5]:
import boto3, cv2, time, numpy as np, matplotlib.pyplot as plt, random
from sagemaker.pytorch import PyTorchPredictor
from sagemaker.deserializers import JSONDeserializer

In [8]:
sm_client = boto3.client(service_name="sagemaker")

# Restore the endpoint name stored in the 2_DeployEndpoint.ipynb notebook
%store -r ENDPOINT_NAME
print(f'Endpoint Name: {ENDPOINT_NAME}')


Recently Created Endpoint Name: x-beagle-deployModel-1716535283133-Endpoint-20240524-072712


In [9]:
import boto3

# SageMaker 클라이언트 생성
sm_client = boto3.client('sagemaker')

# 최근 생성된 엔드포인트 목록 조회
response = sm_client.list_endpoints(
    SortBy='CreationTime',
    SortOrder='Descending'
)

# 최근 생성된 엔드포인트 이름 출력
if response['Endpoints']:
    latest_endpoint_name = response['Endpoints'][0]['EndpointName']
    print(f'Recently Created Endpoint Name: {latest_endpoint_name}')
else:
    print('No endpoints found.')


Recently Created Endpoint Name: x-beagle-deployModel-1716535283133-Endpoint-20240524-072712


In [10]:
sm_client = boto3.client(service_name="sagemaker")

# Restore the endpoint name stored in the 2_DeployEndpoint.ipynb notebook
%store -r latest_endpoint_name
print(f'Endpoint Name: {latest_endpoint_name}')


no stored variable or alias latest_endpoint_name
Endpoint Name: x-beagle-deployModel-1716535283133-Endpoint-20240524-072712


In [11]:
endpoint_created = False
while True:
    response = sm_client.list_endpoints()
    for ep in response['Endpoints']:
        print(f"Endpoint Status = {ep['EndpointStatus']}")
        if ep['EndpointName']==latest_endpoint_name and ep['EndpointStatus']=='InService':
            endpoint_created = True
            break
    if endpoint_created:
        break
    time.sleep(5)

Endpoint Status = InService


In [12]:
predictor = PyTorchPredictor(endpoint_name=latest_endpoint_name,
                             deserializer=JSONDeserializer())

In [13]:
infer_start_time = time.time()

orig_image = cv2.imread('baggage.jpg')

image_height, image_width, _ = orig_image.shape
model_height, model_width = 300, 300
x_ratio = image_width/model_width
y_ratio = image_height/model_height

resized_image = cv2.resize(orig_image, (model_height, model_width))
payload = cv2.imencode('.jpg', resized_image)[1].tobytes()
result = predictor.predict(payload)

infer_end_time = time.time()

print(f"Inference Time = {infer_end_time - infer_start_time:0.4f} seconds")

if 'boxes' in result:
    for idx,(x1,y1,x2,y2,conf,lbl) in enumerate(result['boxes']):
        # Draw Bounding Boxes
        x1, x2 = int(x_ratio*x1), int(x_ratio*x2)
        y1, y2 = int(y_ratio*y1), int(y_ratio*y2)
        color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
        cv2.rectangle(orig_image, (x1,y1), (x2,y2), color, 4)
        cv2.putText(orig_image, f"Class: {int(lbl)}", (x1,y1-40), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
        cv2.putText(orig_image, f"Conf: {int(conf*100)}", (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
        if 'masks' in result:
            # Draw Masks
            mask = cv2.resize(np.asarray(result['masks'][idx]), dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
            for c in range(3):
                orig_image[:,:,c] = np.where(mask>0.5, orig_image[:,:,c]*(0.5)+0.5*color[c], orig_image[:,:,c])

if 'probs' in result:
    # Find Class
    lbl = result['probs'].index(max(result['probs']))
    color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
    cv2.putText(orig_image, f"Class: {int(lbl)}", (20,20), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
    
if 'keypoints' in result:
    # Define the colors for the keypoints and lines
    keypoint_color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
    line_color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))

    # Define the keypoints and the lines to draw
    # keypoints = keypoints_array[:, :, :2]  # Ignore the visibility values
    lines = [
        (0, 1), (0, 2), (1, 3), (2, 4),  # Head
        (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # Torso
        (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)  # Legs
    ]

    # Draw the keypoints and the lines on the image
    for keypoints_instance in result['keypoints']:
        # Draw the keypoints
        for keypoint in keypoints_instance:
            if keypoint[2] == 0:  # If the keypoint is not visible, skip it
                continue
            cv2.circle(orig_image, (int(x_ratio*keypoint[:2][0]),int(y_ratio*keypoint[:2][1])), radius=5, color=keypoint_color, thickness=-1)

        # Draw the lines
        for line in lines:
            start_keypoint = keypoints_instance[line[0]]
            end_keypoint = keypoints_instance[line[1]]
            if start_keypoint[2] == 0 or end_keypoint[2] == 0:  # If any of the keypoints is not visible, skip the line
                continue
            cv2.line(orig_image, (int(x_ratio*start_keypoint[:2][0]),int(y_ratio*start_keypoint[:2][1])),(int(x_ratio*end_keypoint[:2][0]),int(y_ratio*end_keypoint[:2][1])), color=line_color, thickness=2)

plt.imshow(cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB))
plt.show()

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Inference Component Name header is required for endpoints to which you plan to deploy inference components. Please include Inference Component Name header or consider using SageMaker models.

In [19]:
import boto3
import cv2
import time
import json

# SageMaker 클라이언트 초기화
runtime_client = boto3.client('sagemaker-runtime', region_name='us-west-2')

# 엔드포인트 이름
endpoint_name = 'x-beagle-deployModel-1716535283133-Endpoint-20240524-072712'

# 모델 구성 요소 이름 설정
inference_component_name = 'inference-component-x-beagle'  # 실제 구성 요소 이름으로 변경

# 이미지 로드 및 전처리
infer_start_time = time.time()
orig_image = cv2.imread('baggage.jpg')
image_height, image_width, _ = orig_image.shape
model_height, model_width = 300, 300
x_ratio = image_width / model_width
y_ratio = image_height / model_height

resized_image = cv2.resize(orig_image, (model_height, model_width))
_, encoded_image = cv2.imencode('.jpg', resized_image)
payload = encoded_image.tobytes()

# 추론 요청
headers = {
    'Inference-Component-Name': inference_component_name
}
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='image/jpeg',
    Body=payload,
    CustomAttributes='InferenceComponentName={}'.format(inference_component_name)
)

infer_end_time = time.time()
print(f"Inference Time = {infer_end_time - infer_start_time:0.4f} seconds")

# 응답 처리
response_body = response['Body'].read().decode('utf-8')
result = json.loads(response_body)

print("Inference Result:", result)


ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint x-beagle-deployModel-1716535283133-Endpoint-20240524-072712 of account 907729080149 not found.

In [ ]:
response = sm_client.describe_endpoint_config(EndpointConfigName=ENDPOINT_NAME)
print(response)
endpoint_config_name = response['EndpointConfigName']

# Delete Endpoint
sm_client.delete_endpoint(EndpointName=ENDPOINT_NAME)

# Delete Endpoint Configuration
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

# Delete Model
for prod_var in response['ProductionVariants']:
    model_name = prod_var['ModelName']
    sm_client.delete_model(ModelName=model_name)     

In [37]:
import json
import requests
import numpy as np

# 빈 이미지 생성
modelHeight, modelWidth = 640, 640
blank_image = np.zeros((modelHeight, modelWidth, 3), np.uint8)
data = np.array(blank_image.astype(np.float32) / 255.)

# JSON 페이로드 생성
payload = json.dumps([data.tolist()])

# 엔드포인트 URL
endpoint_url = "https://runtime.sagemaker.ap-northeast-2.amazonaws.com/endpoints/x-beagle-deployModel-1716535283133-Endpoint-20240524-072712/invocations"

# 추론 요청 전송
headers = {"Content-Type": "application/json"}
response = requests.post(endpoint_url, data=payload, headers=headers)

# 추론 결과 확인
if response.status_code == 200:
    result = json.loads(response.content.decode("utf-8"))
    print("추론 결과:", result)
else:
    print("추론 요청 실패:", response.status_code, response.content)

추론 요청 실패: 403 b'{"message":"Missing Authentication Token"}'


In [40]:
import json
import boto3
import numpy as np
from botocore.config import Config

# 빈 이미지 생성
modelHeight, modelWidth = 640, 640
blank_image = np.zeros((modelHeight, modelWidth, 3), np.uint8)
data = np.array(blank_image.astype(np.float32) / 255.)

# JSON 페이로드 생성
payload = json.dumps([data.tolist()])

# SageMaker 런타임 클라이언트 생성
runtime = boto3.client('sagemaker-runtime')

# 엔드포인트 이름
endpoint_name = "x-beagle-deployModel-1716535283133-Endpoint-20240524-072712"

# HTTP 헤더 설정
headers = {
    'Content-Type': 'application/json',
    'X-Amzn-SageMaker-Inference-Component-Name': 'x-beagle-deployModel-1716535283133-Endpoint-20240524-072712'  # Inference Component Name 헤더 추가
}

# 추론 요청 전송
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=payload,
    CustomAttributes="X-Amzn-SageMaker-Inference-Component-Name=inference-component-x-beagle"  # Inference Component Name 추가
)

# 추론 결과 확인
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    result = json.loads(response['Body'].read().decode())
    print("추론 결과:", result)
else:
    print("추론 요청 실패:", response['ResponseMetadata']['HTTPStatusCode'])


ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Inference Component Name header is required for endpoints to which you plan to deploy inference components. Please include Inference Component Name header or consider using SageMaker models.

In [25]:
pip install boto3


Note: you may need to restart the kernel to use updated packages.


In [27]:
import numpy as np

ENDPOINT_NAME = 'x-beagle-deployModel-1716535283133-Endpoint-20240524-072712'

modelHeight, modelWidth = 640, 640
blank_image = np.zeros((modelHeight,modelWidth,3), np.uint8)
data = np.array(blank_image.astype(np.float32)/255.)
payload = json.dumps([data.tolist()])
response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
ContentType='application/json',
Body=payload)

result = json.loads(response['Body'].read().decode())
print('Results: ', result)

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Inference Component Name header is required for endpoints to which you plan to deploy inference components. Please include Inference Component Name header or consider using SageMaker models.

In [42]:
pip install opencv-python-headless sagemaker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 20.5 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [49]:
import cv2
import sagemaker
import numpy as np
from sagemaker.predictor import Predictor
from sagemaker.serializers import NumpySerializer

# SageMaker 세션 설정
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

# 엔드포인트 이름 설정 (배포한 엔드포인트 이름을 사용)
endpoint_name = 'x-beagle-deployModel-1716535283133-Endpoint-20240524-072712'

# Predictor 객체 생성
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=NumpySerializer()
)

# 동영상 파일 경로 설정
video_path = 'airport1.mp4'

# 비디오 캡처 객체 생성
cap = cv2.VideoCapture(video_path)

# 동영상 프레임을 읽고 처리
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임 전처리 (예: 크기 조정, 색상 변환 등)
    resized_frame = cv2.resize(frame, (224, 224))  # 모델 입력 크기에 맞게 조정
    input_data = np.expand_dims(resized_frame, axis=0)

    # 예측 요청 보내기
    result = predictor.predict(input_data)
    print(result)

# 비디오 캡처 객체 해제
cap.release()
cv2.destroyAllWindows()


ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Inference Component Name header is required for endpoints to which you plan to deploy inference components. Please include Inference Component Name header or consider using SageMaker models.

In [57]:
import boto3
import cv2
import json
import numpy as np

# SageMaker 런타임 클라이언트 생성
runtime = boto3.client('sagemaker-runtime')

# 엔드포인트 이름
endpoint_name = "x-beagle-deployModel-1716535283133-Endpoint-20240524-072712"

# 비디오 파일 경로
video_path = "airport.mp4"

# 비디오 캡처 객체 생성
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    # 프레임 읽어오기
    ret, frame = cap.read()
    if not ret:
        break
    
    # 프레임 전처리 (모델 입력에 맞게 조정)
    resized_frame = cv2.resize(frame, (640, 640))
    normalized_frame = resized_frame.astype(np.float32) / 255.0
    input_data = np.expand_dims(normalized_frame, axis=0)
    
    # JSON 페이로드 생성
    payload = json.dumps({"inputs": input_data.tolist()})
    
    # 추론 요청 전송
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Body=payload
    )
    
    # 응답 상태 코드 출력
    print("응답 상태 코드:", response['ResponseMetadata']['HTTPStatusCode'])
    
    # 추론 결과 확인
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        result = json.loads(response['Body'].read().decode())
        # 결과 처리 로직 작성
        print("추론 결과:", result)
    else:
        print("추론 요청 실패:", response['ResponseMetadata']['HTTPStatusCode'])
 
        
# 비디오 캡처 객체 해제
cap.release()